In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.read_csv("raw_results.tsv", delimiter='\t')

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
my_dpi = 120

In [ ]:
legend = False

# Performance of all attack methods

In [ ]:
methods = ["pgd", "tga", "degree", "random"]
colors = ["royalblue", "orangered", "hotpink", "dimgray"]#, "orangered"] #, ]
shapes = ['-o', '-x', '->', '-+']#,  #, 's']
labels = ["Dyn-PGD", "Greedy", "Degree", "Random"] #[r"$\mathsc{GIN}$", r"$\mathsc{GCN}$"]#, 'Random']#, 'Degree'] #, 'RL-S2V']

sampling = "td"
epsilons = [0.02, 0.06, 0.10, 0.30, 0.50, 0.70, 0.90]

In [ ]:
def quality_plot (df_dm, dataset, model):
    fig = plt.figure(figsize=(1.7, 2.0), dpi=my_dpi)
    plt.xticks(fontsize = 10)
    plt.yticks(fontsize = 10)
    for method, col, shape, label in zip(methods, colors, shapes, labels):
        df_dmm = df_dm.loc[(df_dm["Method"] == method) & 
                           (df_dm["Sampling"] == sampling) & 
                           (df_dm["Epsilon"].isin(epsilons))]
        y_mean = df_dmm.groupby("Epsilon").mean()
        y_std = df_dmm.groupby("Epsilon").std()
        plt.errorbar(epsilons, -y_mean['Perf. Drop %'], yerr=y_std['Perf. Drop %'], 
                     fmt=shape, color=col, linewidth=1, markersize=6, label=label)
    ax = plt.gca()
    ax.set_xticks([0.00, 0.25, 0.50, 0.75, 1.0])
    if legend:
        handles,leglabels = ax.get_legend_handles_labels()
        plt.legend(handles, leglabels, loc='best', ncol=1, fontsize=10, frameon=True)
    plt.xlabel('Epsilon',fontsize=12)
    plt.ylabel('Rel. Drop %',fontsize=12)
    fig.tight_layout()
    plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
            hspace = 0, wspace = 0)
    fig.savefig(f'Plots/{sampling}/results_{dataset}_{model}.pdf', bbox_inches = 'tight', pad_inches = 0.01)
    plt.close()


In [ ]:
dfg = df.groupby(['Dataset', 'Model'])

In [ ]:
for dataset, model in dfg.groups.keys():
    try:
        df_d = dfg.get_group((dataset, model))
        quality_plot(df_d, dataset, model)
    except:
        print (dataset, model)
        pass

In [ ]:
figsize = (1.5*3, 0.2)
fig_leg = plt.figure(figsize=figsize)
ax_leg = fig_leg.add_subplot(111)
# add the legend from the previous axes
ax_leg.legend(*ax.get_legend_handles_labels(), ncol=4, loc='center')
# hide the axes frame and the x/y labels
ax_leg.axis('off')
fig_leg.savefig('Plots/legend.pdf', bbox_inches = 'tight', pad_inches = 0.01)

# PGD performance across models

In [ ]:
models = ["gclstm", "evgcno", "dysat"]
colors = ["limegreen", "mediumorchid", "chocolate"]#, "orangered"] #, "hotpink"]
shapes = ['-v', '-s', '-*']#, '->' #, 's']
labels = ["GCLSTM", "EvolveGCN", "DySAT"] #[r"$\mathsc{GIN}$", r"$\mathsc{GCN}$"]#, 'Random']#, 'Degree'] #, 'RL-S2V']

sampling = "rd"
epsilons = [0.02, 0.06, 0.10, 0.30, 0.50, 0.70, 0.90]
method = "pgd"

In [ ]:
def quality_plot (df_d, dataset):
    fig = plt.figure(figsize=(1.7, 2.0), dpi=my_dpi)
    plt.xticks(fontsize = 10)
    plt.yticks(fontsize = 10)
    for model, col, shape, label in zip(models, colors, shapes, labels):
        df_dm = df_d.loc[(df_d["Model"] == model) &
                         (df_d["Sampling"] == sampling) &
                         (df_d["Epsilon"].isin(epsilons))]
        df_dmm = df_dm.loc[df_dm["Method"] == method]
        y_mean = df_dmm.groupby("Epsilon").mean()
        y_std = df_dmm.groupby("Epsilon").std()
        plt.errorbar(epsilons, -y_mean['Perf. Drop %'], yerr=y_std['Perf. Drop %'], 
                     fmt=shape, color=col, linewidth=1, markersize=6, label=label)
        # plt.plot(df_dmm["Epsilon"], df_dmm["Perb AUCROC"], shape, color=col, 
        #             linewidth=1, markersize=6, label=label)
    ax = plt.gca()
    if legend:
        handles,leglabels = ax.get_legend_handles_labels()
        plt.legend(handles, leglabels, loc='best', ncol=1, fontsize=10, frameon=True)
    plt.xlabel('Epsilon',fontsize=12)
    plt.ylabel("Rel. Drop %",fontsize=12)
    fig.tight_layout()
    plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
            hspace = 0, wspace = 0)
    fig.savefig(f'Plots/{sampling}/results_{dataset}.pdf', bbox_inches = 'tight', pad_inches = 0.01)
    plt.close()
    return ax


In [ ]:
dfg = df.groupby(['Dataset'])

In [ ]:
for dataset in dfg.groups.keys():
    df_d = dfg.get_group(dataset)
    ax = quality_plot(df_d, dataset)

In [ ]:
figsize = (1.5*2.0, 0.2)
fig_leg = plt.figure(figsize=figsize)
ax_leg = fig_leg.add_subplot(111)
# add the legend from the previous axes
ax_leg.legend(*ax.get_legend_handles_labels(), ncol=4, loc='center')
# hide the axes frame and the x/y labels
ax_leg.axis('off')
fig_leg.savefig('Plots/legend_models.pdf', bbox_inches = 'tight', pad_inches = 0.01)

# Feature perturbation

In [ ]:
df = pd.read_csv("raw_results_feat.tsv", delimiter='\t')

In [ ]:
methods = ["pgd", "random"]
colors = ["royalblue", "dimgray"]#, "orangered"] #, ]
shapes = ['-o', '-+']#,  #, 's']
labels = ["Dyn-PGD", "Random"] #[r"$\mathsc{GIN}$", r"$\mathsc{GCN}$"]#, 'Random']#, 'Degree'] #, 'RL-S2V']

sampling = "rd"
epsilons = [0.02, 0.06, 0.10, 0.30, 0.50, 0.70, 0.90]

In [ ]:
def quality_plot (df_dm, dataset, model):
    fig = plt.figure(figsize=(1.7, 2.0), dpi=my_dpi)
    plt.xticks(fontsize = 10)
    plt.yticks(fontsize = 10)
    for method, col, shape, label in zip(methods, colors, shapes, labels):
        df_dmm = df_dm.loc[(df_dm["Method"] == method) & 
                           (df_dm["Sampling"] == sampling) & 
                           (df_dm["Epsilon"].isin(epsilons))]
        y_mean = df_dmm.groupby("Epsilon").mean()
        y_std = df_dmm.groupby("Epsilon").std()
        plt.errorbar(epsilons, -y_mean['Perf. Drop %'], yerr=y_std['Perf. Drop %'], 
                     fmt=shape, color=col, linewidth=1, markersize=6, label=label)
    ax = plt.gca()
    ax.set_xticks([0.00, 0.25, 0.50, 0.75, 1.0])
    if legend:
        handles,leglabels = ax.get_legend_handles_labels()
        plt.legend(handles, leglabels, loc='best', ncol=1, fontsize=10, frameon=True)
    plt.xlabel('Epsilon',fontsize=12)
    plt.ylabel('Rel. Drop %',fontsize=12)
    fig.tight_layout()
    plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
            hspace = 0, wspace = 0)
    fig.savefig(f'Plots/feat/{sampling}/results_{dataset}_{model}.pdf', bbox_inches = 'tight', pad_inches = 0.01)
    plt.close()
    return ax

In [ ]:
dfg = df.groupby(['Dataset', 'Model'])

In [ ]:
for dataset, model in dfg.groups.keys():
    df_d = dfg.get_group((dataset, model))
    ax = quality_plot(df_d, dataset, model)

In [ ]:
figsize = (1.5*2, 0.2)
fig_leg = plt.figure(figsize=figsize)
ax_leg = fig_leg.add_subplot(111)
# add the legend from the previous axes
ax_leg.legend(*ax.get_legend_handles_labels(), ncol=4, loc='center')
# hide the axes frame and the x/y labels
ax_leg.axis('off')
fig_leg.savefig('Plots/legend_feat.pdf', bbox_inches = 'tight', pad_inches = 0.01)

# Online perturbation

In [ ]:
df = pd.read_csv("raw_results_online3.tsv", delimiter='\t')

In [ ]:
methods = ["pgd", "degree", "random"]
colors = ["royalblue", "hotpink", "dimgray"]#, "orangered"] #, ]
shapes = ['-o', '->', '-+']#,  #, 's']
labels = ["Dyn-PGD", "Degree", "Random"] #[r"$\mathsc{GIN}$", r"$\mathsc{GCN}$"]#, 'Random']#, 'Degree'] #, 'RL-S2V']

sampling = "rd"
epsilons = [0.02, 0.06, 0.10, 0.30, 0.50, 0.70, 0.90]

In [ ]:
def quality_plot (df_dm, dataset, model):
    fig = plt.figure(figsize=(1.7, 2.0), dpi=my_dpi)
    plt.xticks(fontsize = 10)
    plt.yticks(fontsize = 10)
    for method, col, shape, label in zip(methods, colors, shapes, labels):
        df_dmm = df_dm.loc[(df_dm["Method"] == method) & 
                           (df_dm["Sampling"] == sampling) & 
                           (df_dm["Epsilon"].isin(epsilons))]
        y_mean = df_dmm.groupby("Epsilon").mean()
        y_std = df_dmm.groupby("Epsilon").std()
        plt.errorbar(epsilons, -y_mean['Perf. Drop %'], yerr=y_std['Perf. Drop %'], 
                     fmt=shape, color=col, linewidth=1, markersize=6, label=label)
    ax = plt.gca()
    ax.set_xticks([0.00, 0.25, 0.50, 0.75, 1.0])
    if legend:
        handles,leglabels = ax.get_legend_handles_labels()
        plt.legend(handles, leglabels, loc='best', ncol=1, fontsize=10, frameon=True)
    plt.xlabel('Epsilon',fontsize=12)
    plt.ylabel('Rel. Drop %',fontsize=12)
    fig.tight_layout()
    plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
            hspace = 0, wspace = 0)
    fig.savefig(f'Plots/online/{sampling}/results_{dataset}_{model}.pdf', bbox_inches = 'tight', pad_inches = 0.01)
    plt.close()
    return ax

In [ ]:
dfg = df.groupby(['Dataset', 'Model'])

In [ ]:
for dataset, model in dfg.groups.keys():
    try:
        df_dm = dfg.get_group((dataset, model))
        ax = quality_plot(df_dm, dataset, model)
    except:
        print (dataset, model)
        pass

In [ ]:
figsize = (1.5*2, 0.2)
fig_leg = plt.figure(figsize=figsize)
ax_leg = fig_leg.add_subplot(111)
# add the legend from the previous axes
ax_leg.legend(*ax.get_legend_handles_labels(), ncol=4, loc='center')
# hide the axes frame and the x/y labels
ax_leg.axis('off')
fig_leg.savefig('Plots/legend_online.pdf', bbox_inches = 'tight', pad_inches = 0.01)

# Target-specific metric plots

In [ ]:
methods = ["pgd", "tga", "degree", "random"]
colors = ["royalblue", "orangered", "hotpink", "dimgray"]#, "orangered"] #, ]
shapes = ['-o', '-x', '->', '-+']#,  #, 's']
labels = ["Dyn-PGD", "Greedy", "Degree", "Random"] #[r"$\mathsc{GIN}$", r"$\mathsc{GCN}$"]#, 'Random']#, 'Degree'] #, 'RL-S2V']

sampling = "rd"
epsilons = [0.02, 0.06, 0.10, 0.30, 0.50, 0.70, 0.90]
metric = "E"
cohen_d = False

In [ ]:
def quality_plot (df_dm, dataset, model):
    fig = plt.figure(figsize=(1.7, 2.0), dpi=my_dpi)
    plt.xticks(fontsize = 10)
    plt.yticks(fontsize = 10)
    for method, col, shape, label in zip(methods, colors, shapes, labels):
        df_dm = df_dm.dropna(subset=[metric])
        df_dm[f'{metric}_mean'] = [float(x.split(", ")[0]) for x in df_dm[metric]]
        df_dm[f'{metric}_std'] = [float(x.split(", ")[1]) for x in df_dm[metric]]
        df_dmm = df_dm.loc[(df_dm["Method"] == method) & 
                        (df_dm["Sampling"] == sampling) & 
                        (df_dm["Epsilon"].isin(epsilons))]
        y_mean = df_dmm.groupby("Epsilon").mean()
        if (cohen_d):
            y_mean[f'{metric}_cohen'] = (y_mean[f'{metric}_mean'] - 0)/y_mean[f'{metric}_std']
            plt.plot(epsilons, y_mean[f'{metric}_cohen'], shape, color=col, linewidth=1, markersize=6, label=label)
        else:
            plt.errorbar(epsilons, y_mean[f'{metric}_mean'], yerr=y_mean[f'{metric}_std'], 
                        fmt=shape, color=col, linewidth=1, markersize=6, label=label)
    ax = plt.gca()
    ax.set_xticks([0.00, 0.25, 0.50, 0.75, 1.0])
    if legend:
        handles,leglabels = ax.get_legend_handles_labels()
        plt.legend(handles, leglabels, loc='best', ncol=1, fontsize=10, frameon=True)
    plt.xlabel('Epsilon',fontsize=12)
    if cohen_d:
        plt.ylabel(f'{metric}_cohen',fontsize=12)
    else:
        plt.ylabel(f'{metric}',fontsize=12)
    fig.tight_layout()
    plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0)
    if cohen_d:
        fig.savefig(f'Plots/{metric}_cohen/{sampling}/results_{dataset}_{model}.pdf', bbox_inches = 'tight', pad_inches = 0.01)
    else:
        fig.savefig(f'Plots/{metric}/{sampling}/results_{dataset}_{model}.pdf', bbox_inches = 'tight', pad_inches = 0.01)
    plt.close()


In [ ]:
dfg = df.groupby(['Dataset', 'Model'])

In [ ]:
for dataset, model in dfg.groups.keys():
    # try:
    df_d = dfg.get_group((dataset, model))
    quality_plot(df_d, dataset, model)
    # except:
    #     print (dataset, model)
    #     pass

# E metric statistics

In [ ]:
methods = ["pgd", "tga", "degree", "random"]

sampling = "rd"
epsilons = [0.02, 0.06, 0.10, 0.30, 0.50, 0.70, 0.90]
metric = "E"

In [ ]:
def resdata_append (df_dm, dataset, model, res_df):
    for method in methods:
        df_dm = df_dm.dropna(subset=[metric])
        df_dm[f'{metric}_mean'] = [float(x.split(", ")[0]) for x in df_dm[metric]]
        df_dm[f'{metric}_std'] = [float(x.split(", ")[1]) for x in df_dm[metric]]
        df_dmm = df_dm.loc[(df_dm["Method"] == method) & 
                        (df_dm["Sampling"] == sampling) & 
                        (df_dm["Epsilon"].isin(epsilons))]
        y_mean = df_dmm.groupby("Epsilon").mean()
        y_std = df_dmm.groupby("Epsilon").std()
        cohen_d = (y_mean[f'{metric}_mean'] - 0)/y_mean[f'{metric}_std']
        for epsilon in cohen_d.index:
            res_df["Dataset"].append(dataset)
            res_df["Model"].append(model)
            res_df["Method"].append(method)
            res_df["Epsilon"].append(epsilon)
            res_df["Mean Perf. Drop %"].append(y_mean['Perf. Drop %'].loc[epsilon])
            res_df["Std Perf. Drop %"].append(y_std['Perf. Drop %'].loc[epsilon])
            res_df["Cohen_d"].append(cohen_d.loc[epsilon])
            res_df[f'{metric}_mean'].append(y_mean[f'{metric}_mean'].loc[epsilon])
            res_df[f'{metric}_std'].append(y_mean[f'{metric}_std'].loc[epsilon])
    return res_df

In [ ]:
dfg = df.groupby(['Dataset', 'Model'])

In [ ]:
from collections import defaultdict

res_df = defaultdict(lambda: [])

for dataset, model in dfg.groups.keys():
    df_d = dfg.get_group((dataset, model))
    res_df = resdata_append(df_d, dataset, model, res_df)

In [ ]:
res_df = pd.DataFrame(res_df)

In [ ]:
res_df_tab = res_df.loc[(res_df['Epsilon'] == 0.5) & ~(res_df['Dataset'].isin(["DBLP"])) & (res_df["Method"] == 'pgd')]

In [ ]:
res_df_tab[['Dataset', 'Model', 'Mean Perf. Drop %',
       'Std Perf. Drop %', 'Cohen_d', 'E_mean', 'E_std']]

In [ ]:
res_df.to_csv("e.csv", index=False)

# Running Time experiment

In [ ]:
random_reddit_mean = [4.01380466, 4.248379645, 3.971783389, 5.688414611, 6.588495526, 6.306535423, 3.106602286]
random_reddit_std = [1.506646785, 1.344604474, 1.37294442, 2.953934616, 4.031191687, 6.299767277, 2.720999723]

degree_reddit_mean = [4.274370613, 4.57556869, 4.00092563, 4.066166203, 4.419590709, 4.114394008, 4.065688193]
degree_reddit_std = [1.230938658, 1.337842071, 1.442863082, 1.167981418, 1.414808568, 1.258317752, 1.278922633]

In [ ]:
methods = ["pgd", "tga", "degree", "random"]
colors = ["royalblue", "orangered", "hotpink", "dimgray"]#, "orangered"] #, ]
shapes = ['-o', '-x', '->', '-+']#,  #, 's']
labels = ["Dyn-PGD", "Greedy", "Degree", "Random"] #[r"$\mathsc{GIN}$", r"$\mathsc{GCN}$"]#, 'Random']#, 'Degree'] #, 'RL-S2V']

sampling = "rd"
epsilons = [0.02, 0.06, 0.10, 0.30, 0.50, 0.70, 0.90]

In [ ]:
def quality_plot (df_dm, dataset, model):
    fig = plt.figure(figsize=(1.7, 2.0), dpi=my_dpi)
    plt.xticks(fontsize = 10)
    plt.yticks(fontsize = 10)
    for method, col, shape, label in zip(methods, colors, shapes, labels):
        if method == "random" and dataset == "reddit_hyperlinks":
            plt.errorbar(epsilons, random_reddit_mean, yerr=random_reddit_std, 
                     fmt=shape, color=col, linewidth=1, markersize=6, label=label)
        elif method == "degree" and dataset == "reddit_hyperlinks":
            plt.errorbar(epsilons, degree_reddit_mean, yerr=degree_reddit_std, 
                     fmt=shape, color=col, linewidth=1, markersize=6, label=label)
        else:
            df_dmm = df_dm.loc[(df_dm["Method"] == method) & 
                            (df_dm["Sampling"] == sampling) & 
                            (df_dm["Epsilon"].isin(epsilons))]
            df_dmm['Time taken'] = [float(x) for x in df_dmm["Time taken"]]
            y_mean = df_dmm.groupby("Epsilon").mean()
            y_std = df_dmm.groupby("Epsilon").std()
            # print (method, model, dataset, y_mean.columns, y_std.columns)
            plt.errorbar(epsilons, y_mean['Time taken'], yerr=y_std['Time taken'], 
                        fmt=shape, color=col, linewidth=1, markersize=6, label=label)
    ax = plt.gca()
    ax.set_xticks([0.00, 0.25, 0.50, 0.75, 1.0])
    if legend:
        handles,leglabels = ax.get_legend_handles_labels()
        plt.legend(handles, leglabels, loc='best', ncol=1, fontsize=10, frameon=True)
    plt.xlabel('Epsilon',fontsize=12)
    plt.ylabel('Time/target (s)',fontsize=12)
    fig.tight_layout()
    plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
            hspace = 0, wspace = 0)
    fig.savefig(f'Plots/Time/{sampling}/results_{dataset}_{model}.pdf', bbox_inches = 'tight', pad_inches = 0.01)
    plt.close()
    return ax


In [ ]:
dfg = df.groupby(['Dataset', 'Model'])

In [ ]:
for dataset, model in dfg.groups.keys():
    try:
        df_d = dfg.get_group((dataset, model))
        quality_plot(df_d, dataset, model)
    except:
        print (dataset, model)
        pass